# Packages

In [41]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import yfinance as yf
import yahoofinancials
from scipy.optimize import minimize
import seaborn as sn
from scipy.stats import anderson,kstest,shapiro, probplot

# Masalah.  

In [42]:
stock = ['TSLA', 'DIS', 'NKE', 'AAPL', 'AXP', 'GS', 'GD', 'IBM', 'INTC', 'KO', 'PG', 'V', 'MA', 'BAC', 'NVDA', 'GOOGL', 'GOOG', 'FB', 'MSFT', 'AMZN',]
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()

In [43]:
mu = df.mean().values
C = df.cov().values
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [44]:
np.sum(np.round(x_,5))

1.0

In [45]:
port_ = dict(zip(stock,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
port

{'IBM': 0.14274,
 'INTC': 0.19848,
 'KO': 0.13408,
 'PG': 0.31288,
 'BAC': 0.00255,
 'GOOGL': 0.05987,
 'GOOG': 0.05542,
 'MSFT': 0.07396,
 'AMZN': 0.02002}

In [46]:
start = 1e5
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget_

{'IBM': 14274.0,
 'INTC': 19848.0,
 'KO': 13408.0,
 'PG': 31288.0,
 'BAC': 255.00000000000003,
 'GOOGL': 5987.0,
 'GOOG': 5542.0,
 'MSFT': 7396.0,
 'AMZN': 2002.0}

In [47]:
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
budget

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Total Spread,14274.0,19848.0,13408.0,31288.0,255.0,5987.0,5542.0,7396.0,2002.0


In [48]:
expense = pd.DataFrame(columns=port.keys())
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
expense

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Date,,,,,,,,,
Closing 2022-05-20,12641.500092,4058.499908,6027.500153,14017.999268,3323.500061,212753.491211,213855.493164,24917.900085,211351.000977


In [49]:
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Closing 2022-05-20,12641.500092,4058.499908,6027.500153,14017.999268,3323.500061,212753.491211,213855.493164,24917.900085,211351.000977
Total Spread,14274.000000,19848.000000,13408.000000,31288.000000,255.000000,5987.000000,5542.000000,7396.000000,2002.000000
Total Lot,1.000000,5.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Total Spending,12642.000000,20292.000000,12055.000000,28036.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [50]:
finale.sum(axis = 1)

Closing 2022-05-20    702946.884918
Total Spread          100000.000000
Total Lot                 10.000000
Total Spending         73025.000000
dtype: float64

In [51]:
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

return : 0.4982235658699521 %
risk : 0.14348467534309714 %
Total Money : 73025.0
Total Idle Money : 26975.0


In [52]:
finale.to_csv(str('USA '+finale.index[0].split(' ')[1]+'.csv'))

# Testing Profit

In [53]:
year = 2010
ds = str(year-1)+'-12-01'
de = str(year+2)+'-12-01'

stock = ['TSLA', 'DIS', 'NKE', 'AAPL', 'AXP', 'GS', 'GD', 'IBM', 'INTC', 'KO', 'PG', 'V', 'MA', 'BAC', 'NVDA', 'GOOGL', 'GOOG', 'FB', 'MSFT', 'AMZN',]
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()
df.dropna(axis = 1, inplace = True)

In [54]:
mu = df.mean().values
C = df.cov().values

def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [55]:
print(np.sum(np.round(x_,5)))
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

1.00001
return : 1.5796279386643433 %
risk : 0.06206158049243347 %
Total Money : 73025.0
Total Idle Money : 26975.0


In [56]:
port_ = dict(zip(df.columns.values,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
start = 1e5
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
expense = pd.DataFrame(columns=port.keys())

bs = str(year+2)+'-12-01'
be = str(year+3)+'-01-01'
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,TSLA,DIS,IBM,INTC,KO,PG,V,AMZN
Closing 2012-11-01,6.764000e+02,4.446277e+03,1.228011e+04,1513.710594,2.830713e+03,5.309067e+03,3.495870e+03,2.520500e+04
2012-12-01 00:00:00,6.774000e+02,4.457916e+03,1.243239e+04,1611.361313,2.724615e+03,5.161571e+03,3.547755e+03,2.508700e+04
Total Spread,3.288000e+03,2.516000e+03,1.299200e+04,351.000000,2.702600e+04,3.643400e+04,9.431000e+03,7.963000e+03
Total Lot,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Total Spending,2.224003e+06,1.118683e+07,1.595432e+08,531312.000000,7.650284e+07,1.934306e+08,3.296955e+07,2.007074e+08


In [57]:
buy = finale.columns.values
bs = str(year+2)+'-12-01'
be = str(year+4)+'-01-01'
profit_test = pd.DataFrame()
for item in buy:
    temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
    temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
    profit_test['buy_'+item] = temp['buy_'+item].dropna()
    profit_test['sell_'+item] = temp['sell_'+item].dropna()
    profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
profit_test.dropna(inplace = True)
profit_test


,buy_TSLA,sell_TSLA,return_TSLA,buy_DIS,sell_DIS,return_DIS,buy_IBM,sell_IBM,return_IBM,buy_INTC,...,return_KO,buy_PG,sell_PG,return_PG,buy_V,sell_V,return_V,buy_AMZN,sell_AMZN,return_AMZN
Date,,,,,,,,,,,,,,,,,,,,,
2012-12-01,677.799988,677.400017,-0.399971,4456.125531,4457.916260,1.790728,12381.120729,12432.394409,51.273680,1553.533474,...,-124.017069,5309.826358,5161.571121,-148.255237,3510.774846,3547.755051,36.980205,25253.999329,25086.999512,-166.999817
2013-01-01,700.000000,750.199986,50.199986,4618.191819,4898.192596,280.000777,12597.248130,13180.087280,582.839151,1652.779072,...,18.790456,5219.351860,5714.297104,494.945244,3608.139019,3695.908356,87.769337,25607.998657,26550.000000,942.001343
2013-02-01,763.399982,696.600008,-66.799974,4925.465243,4962.738037,37.272794,13282.638543,13034.704590,-247.933954,1655.905158,...,88.690952,5771.356239,5838.803482,67.447243,3721.420639,3712.994766,-8.425872,26892.999268,26426.998901,-466.000366
2013-03-01,700.000000,757.800007,57.800007,4937.283165,5163.647079,226.363915,13080.371438,13905.024719,824.653282,1642.107858,...,146.565339,5802.014116,5906.251144,104.237028,3704.173197,3983.516693,279.343496,26326.998901,26648.999023,322.000122
2013-04-01,847.200012,1079.800034,232.600021,5171.829586,5712.739944,540.910359,13872.431942,13203.582764,-668.849178,1711.614808,...,146.869638,5890.154630,5884.022903,-6.131726,3981.406701,3951.150513,-30.256188,26698.001099,25380.999756,-1317.001343
2013-05-01,1119.799995,1955.200005,835.400009,5716.376178,5734.558105,18.181927,13159.905200,13560.823059,400.917859,1881.433190,...,-163.524728,5931.888070,5926.483536,-5.404534,3916.669487,4178.187180,261.517692,25389.999390,26920.001221,1530.001831
2013-06-01,1952.400017,2147.200012,194.799995,5732.739934,5740.921783,8.181850,13642.391414,12519.555664,-1122.835750,1983.670199,...,12.112932,5934.975992,5944.240570,9.264578,4200.183688,4294.173813,93.990124,26895.999146,27769.000244,873.001099
2013-07-01,2187.199974,2685.600090,498.400116,5803.648740,5877.285385,73.636645,12587.691200,12777.013397,189.322197,1941.413189,...,-32.016115,5946.557844,6199.800491,253.242647,4321.195129,4159.297180,-161.897949,27900.000000,30122.000122,2222.000122
2013-08-01,2700.000000,3379.999924,679.999924,5923.649512,5530.012131,-393.637382,12882.476701,11940.447998,-942.028703,1866.467179,...,-169.228841,6348.916114,6058.773041,-290.143073,4270.910408,4098.438644,-172.471763,30307.998657,28098.001099,-2209.997559


In [58]:
cols = profit_test.columns
finale_pt = pd.DataFrame()
tspend = 0
tprofit = 0 
for k in range (0,profit_test.shape[1], 3):
    tspend += profit_test[cols[k]]
    tprofit += profit_test[cols[k+2]]
finale_pt['Total Spending'] = tspend
finale_pt['Total Profit'] = tprofit
finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
finale_pt

,Total Spending,Total Profit,Total Return
Date,,,
2012-12-01,55991.812231,-291.799642,-0.005211
2013-01-01,56783.944589,2447.950380,0.043110
2013-02-01,59834.759278,-619.974418,-0.010361
2013-03-01,59085.927631,2043.897866,0.034592
2013-04-01,61230.400181,-921.771526,-0.015054
2013-05-01,60307.075219,2913.423438,0.048310
2013-06-01,63366.811139,16.690677,0.000263
2013-07-01,63774.987809,2962.160857,0.046447
2013-08-01,67380.076986,-3611.520582,-0.053599


In [59]:
print(f'Keuntungan dalam satu tahun pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

Keuntungan dalam satu tahun pada tahun 2013 sebesar: 2.1315350867991705 %


# Loop

In [60]:
year = 2010
for n in range(8):
    year += 1
    ds = str(year-1)+'-12-01'
    de = str(year+2)+'-12-01'

    stock = ['TSLA', 'DIS', 'NKE', 'AAPL', 'AXP', 'GS', 'GD', 'IBM', 'INTC', 'KO', 'PG', 'V', 'MA', 'BAC', 'NVDA', 'GOOGL', 'GOOG', 'FB', 'MSFT', 'AMZN',]
    df = pd.DataFrame(columns=stock)
    for i in stock :
        temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
        temp['log_return'].dropna(inplace = True)
        df[i] =temp['log_return'].dropna()
    df.dropna(axis = 1, inplace = True)

    mu = df.mean().values
    C = df.cov().values

    def objective(weights): 
        weights = np.array(weights)
        return weights.dot(C).dot(weights.T)
    # The constraints
    cons = (# The weights must sum up to one.
            {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
            # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
            #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

            )
    # Every stock can get any weight from 0 to 1
    bounds = tuple((0,1) for x in range(mu.shape[0]))
    # Initialize the weights with an even split
    # In out case each stock will have 10% at the beginning
    guess = [1./mu.shape[0] for x in range(mu.shape[0])]
    optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
    x_ = optimized_results.x
    print(np.sum(np.round(x_,5)))
    print(f'return :', x_.dot(mu)*100, '%')
    print(f'risk :', x_.dot(C).dot(x_)*100, '%')
    print(f'Total Money :',finale.sum(axis=1)[-1])
    print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])
    port_ = dict(zip(df.columns.values,np.round(x_,5)))
    port = dict()
    for (key,value) in port_.items():
        if value >0 : 
            port[key] = value
    start = 1e5 
    budget_ = dict()
    for (key,value) in port.items():
        if value >0 : 
            budget_[key] = value*start
    budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
    budget.rename(index={0: 'Total Spread'}, inplace= True)
    expense = pd.DataFrame(columns=port.keys())

    bs = str(year+2)+'-12-01'
    be = str(year+3)+'-01-01'

    for item in port.keys(): 
        temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
        expense[item] = temp['Close']*100 #1 lot
    expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
    qty = pd.concat([expense,budget])
    temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
    temp_.rename(index={0: 'Total Lot'}, inplace= True)
    qty = pd.concat([qty, temp_])
    spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
    finale = pd.concat([qty, spend])
    finale.rename(index={0: 'Total Spending'}, inplace= True)
    finale
    buy = finale.columns.values
    bs = str(year+2)+'-12-01'
    be = str(year+4)+'-01-01'
    profit_test = pd.DataFrame()
    for item in buy:
        temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
        temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
        profit_test['buy_'+item] = temp['buy_'+item].dropna()
        profit_test['sell_'+item] = temp['sell_'+item].dropna()
        profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
    profit_test.dropna(inplace = True)
    cols = profit_test.columns
    finale_pt = pd.DataFrame()
    tspend = 0
    tprofit = 0 
    for k in range (0,profit_test.shape[1], 3):
        tspend += profit_test[cols[k]]
        tprofit += profit_test[cols[k+2]]
    finale_pt['Total Spending'] = tspend
    finale_pt['Total Profit'] = tprofit
    finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
    finale_pt.to_csv('Profit Test USA' +str(year)+'.csv')
    print(f'Rata-rata satu tahun keuntungan perbulan pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

1.0000200000000001
return : 1.5194388893595552 %
risk : 0.050850058417170746 %
Total Money : 677095684.0
Total Idle Money : -676995684.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2014 sebesar: 1.2188597659721663 %
1.00001
return : 1.5912188373704732 %
risk : 0.0500058124300588 %
Total Money : 96915.0
Total Idle Money : 3085.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2015 sebesar: 0.29246547512466914 %
1.00002
return : 1.1651405593842616 %
risk : 0.0725883567045571 %
Total Money : 93232.0
Total Idle Money : 6768.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2016 sebesar: 0.9424684659442135 %
1.0
return : 0.769074582627414 %
risk : 0.06632858406946263 %
Total Money : 101535.0
Total Idle Money : -1535.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2017 sebesar: 1.1117990066740855 %
0.9999899999999999
return : 1.0563558109170053 %
risk : 0.05441697286279716 %
Total Money : 89091.0
Total Idle Money : 10909.0
Rata-rata satu tahun keuntungan perbulan pada t